In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [44]:
import pandas as pd
import time
from selenium.webdriver.common.by import By
driver.get('https://www.lusha.com/company-search/software-development/021d3a8f4f/iran-islamic-republic-of/b50af49bc6/')

# Extract English company names
english_company_names = []
for i in range(0, 50):
    try:
        company_css_selector = f'section.directory-content:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child({(i//3)+1}) > div:nth-child({(i%3)+1}) > a:nth-child(1)'
        company_element =driver.find_element(By.CSS_SELECTOR,company_css_selector)
        print(i)
        english_company_names.append(company_element.text)
    except:
        print("error",i)
        pass

# Clean up
driver.quit()

# Create DataFrame
df = pd.DataFrame({'English Company Name': english_company_names})

# Save to Excel file
df.to_excel('software_engineering_companies.xlsx', index=False)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [45]:
df

,English Company Name
0,0&1 Information Technology Co.
1,2A Company
2,7030.ir
3,ACACO
4,AKAF SYSTEM
5,ARIISCO
6,ARYANIC
7,Abarsazeha
8,Adak Software Company
9,Aien Co


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Load company names from Excel file
companies_df = pd.read_excel('software_engineering_companies.xlsx')
companies = companies_df['English Company Name'].tolist()

# Set up Selenium WebDriver for Firefox
geckodriver_path = '/path/to/geckodriver'
driver = webdriver.Firefox(executable_path=geckodriver_path)
driver.get('https://www.linkedin.com/login')

# Log in to LinkedIn (manual interaction required)
input("Press Enter after logging in...")

# Create a list to store the data
data = []

for company in companies:
    search_url = f"https://www.linkedin.com/search/results/people/?currentCompany=%5B{company}%5D&keywords=software%20engineer&origin=GLOBAL_SEARCH_HEADER"
    driver.get(search_url)
    time.sleep(5)  # Wait for the page to load

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    profiles = soup.select('.search-result__info')

    for profile in profiles:
        try:
            full_name = profile.find('span', class_='actor-name').text.strip()
            headline = profile.find('p', class_='subline-level-1').text.strip()
            location = profile.find('p', class_='subline-level-2').text.strip()

            # Click on the profile link to get more details
            profile_link = profile.find('a')['href']
            driver.get(f"https://www.linkedin.com{profile_link}")
            time.sleep(5)  # Wait for the page to load

            profile_soup = BeautifulSoup(driver.page_source, 'html.parser')
            experiences = profile_soup.select('.pv-entity__summary-info')
            work_experiences = [exp.find('h3').text.strip() for exp in experiences[:3]]
            education_section = profile_soup.select('.pv-education-entity')
            education_qualifications = [edu.find('h3').text.strip() for edu in education_section[:3]]
            skills = profile_soup.select('.pv-skill-category-entity__name-text')
            key_skills = [skill.text.strip() for skill in skills[:5]]

            # Collect the data
            data.append({
                'Full name': full_name,
                'Email': None,  # LinkedIn usually does not provide emails
                'Website': None,  # Custom scraping logic needed for personal websites
                'LinkedIn headline': headline,
                'City and country of residence': location,
                'Current company': company,
                'Three work experiences': work_experiences,
                'Three educational qualifications': education_qualifications,
                'Five key skills': key_skills
            })

        except Exception as e:
            print(f"An error occurred: {e}")

# Clean up
driver.quit()

# Create DataFrame
df = pd.DataFrame(data)

# Save to Excel file
df.to_excel('software_engineering_profiles.xlsx', index=False)


In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

class LinkedInProfileSpider(scrapy.Spider):
    name = 'linkedin_profiles'
    start_urls = ['https://www.linkedin.com/jobs/search?keywords=Software%20Engineer&location=Iran']

    def parse(self, response):
        profiles = response.css('.result-card__full-card-link::attr(href)').getall()
        for profile in profiles:
            yield scrapy.Request(profile, callback=self.parse_profile)

    def parse_profile(self, response):
        full_name = response.css('.top-card-layout__title::text').get().strip()
        headline = response.css('.top-card-layout__headline::text').get().strip()
        location = response.css('.top-card-layout__first-subline::text').get().strip()

        work_experiences = response.css('.experience-item__title::text').getall()[:3]
        education_qualifications = response.css('.education__item__degree-info::text').getall()[:3]
        key_skills = response.css('.skill::text').getall()[:5]

        yield {
            'Full name': full_name,
            'Email': None,
            'Website': None,
            'LinkedIn headline': headline,
            'City and country of residence': location,
            'Current company': None,
            'Three work experiences': work_experiences,
            'Three educational qualifications': education_qualifications,
            'Five key skills': key_skills
        }

# Run Scrapy spider
process = CrawlerProcess(settings={
    'FEED_FORMAT': 'json',
    'FEED_URI': 'linkedin_profiles.json'
})
process.crawl(LinkedInProfileSpider)
process.start()

# Load JSON and convert to DataFrame
profiles_df = pd.read_json('linkedin_profiles.json')
profiles_df.to_excel('software_engineering_profiles_scrapy.xlsx', index=False)


In [ ]:
import requests
import pandas as pd

# Load company names from Excel file
companies_df = pd.read_excel('software_engineering_companies.xlsx')
companies = companies_df['English Company Name'].tolist()

# Define API key and endpoint
api_key = 'your_fake_api_key'
api_endpoint = 'https://nubela.co/proxycurl/api/v2/linkedin'

headers = {
    'Authorization': f'Bearer {api_key}',
}

# Create a list to store the data
data = []

for company in companies:
    params = {
        'url': 'https://www.linkedin.com/in/amir-fesharaki-244a71290/',
        'use_cache': 'if-present',
    }
    response = requests.get(api_endpoint, headers=headers, params=params)
    profile_data = response.json()

    if response.status_code == 200 and 'data' in profile_data:
        profile = profile_data['data']
        data.append({
            'Full name': profile.get('full_name'),
            'Email': profile.get('email'),
            'Website': profile.get('website'),
            'LinkedIn headline': profile.get('headline'),
            'City and country of residence': profile.get('location'),
            'Current company': profile.get('current_company'),
            'Three work experiences': profile.get('experiences', [])[:3],
            'Three educational qualifications': profile.get('education', [])[:3],
            'Five key skills': profile.get('skills', [])[:5]
        })

# Create DataFrame
df = pd.DataFrame(data)

# Save to Excel file
df.to_excel('software_engineering_profiles_proxycurl.xlsx', index=False)


In [ ]:
import requests
import pandas as pd

# Load company names from Excel file
companies_df = pd.read_excel('software_engineering_companies.xlsx')
companies = companies_df['English Company Name'].tolist()

# Define API key and endpoint
api_key = 'your_fake_api_key'
api_endpoint = 'https://api.example.com/profiles'

headers = {
    'Authorization': f'Bearer {api_key}',
}

# Create a list to store the data
data = []

for company in companies:
    params = {
        'company': company,
        'keywords': 'software engineer',
        'location': 'Iran'
    }
    response = requests.get(api_endpoint, headers=headers, params=params)
    profiles_data = response.json()

    if response.status_code == 200:
        for profile in profiles_data['profiles']:
            data.append({
                'Full name': profile.get('full_name'),
                'Email': profile.get('email'),
                'Website': profile.get('website'),
                'LinkedIn headline': profile.get('headline'),
                'City and country of residence': profile.get('location'),
                'Current company': company,
                'Three work experiences': profile.get('experiences', [])[:3],
                'Three educational qualifications': profile.get('education', [])[:3],
                'Five key skills': profile.get('skills', [])[:5]
            })

# Create DataFrame
df = pd.DataFrame(data)

# Save to Excel file
df.to_excel('software_engineering_profiles_api.xlsx', index=False)


**LLM SCRAPING**


In [ ]:
from scrapegraphai import ScrapeGraph
import pandas as pd

# Initialize ScrapeGraph with a generated API key
api_key = '123e4567-e89b-12d3-a456-426614174000'
scrapegraph = ScrapeGraph(api_key=api_key)


In [ ]:
# Load company names from Excel file
companies_df = pd.read_excel('software_engineering_companies.xlsx')
companies = companies_df['English Company Name'].tolist()

# Define the scraping prompt for ScrapeGraphAI
prompt = """
Create a web scraper to collect data on active individuals in the "Software Engineering" field in Iran. 
These individuals should currently work at one of the top 50 domestic companies.
The dataset should include at least the following features for each person:
- Full name
- Email (if available)
- Website (if available)
- LinkedIn headline
- City and country of residence
- Current company
- Three work experiences
- Three educational qualifications
- Five key skills
Aim for a dataset with at least 2000 records.
"""

# Function to collect data for each company
def collect_data_for_company(company_name):
    query = {
        "company": company_name,
        "keywords": "Software Engineer",
        "location": "Iran"
    }
    response = scrapegraph.run(prompt, query)
    return response

# Create a list to store the collected data
data = []

# Collect data for each company
for company in companies:
    try:
        company_data = collect_data_for_company(company)
        for profile in company_data:
            data.append({
                'Full name': profile.get('full_name'),
                'Email': profile.get('email'),
                'Website': profile.get('website'),
                'LinkedIn headline': profile.get('headline'),
                'City and country of residence': profile.get('location'),
                'Current company': company,
                'Three work experiences': profile.get('experiences', [])[:3],
                'Three educational qualifications': profile.get('education', [])[:3],
                'Five key skills': profile.get('skills', [])[:5]
            })
    except Exception as e:
        print(f"An error occurred for {company}: {e}")

# Create DataFrame
df = pd.DataFrame(data)

# Save to Excel file
df.to_excel('software_engineering_profiles_scrapegraphai.xlsx', index=False)


In [ ]:
# Define the generation prompt for ScrapeGraphAI
generation_prompt = """
Generate a dataset with a minimum of 100 records based on the following input criteria:
- Field of activity
- Country
- University
- Current company
"""

# Function to generate dataset based on input criteria
def generate_dataset(field_of_activity, country, university, current_company):
    query = {
        "field_of_activity": field_of_activity,
        "country": country,
        "university": university,
        "current_company": current_company
    }
    response = scrapegraph.run(generation_prompt, query)
    return response

# Example input criteria
field_of_activity = "Software Engineering"
country = "Iran"
university = "University of Tehran"
current_company = "Snapp"

# Generate dataset
dataset = generate_dataset(field_of_activity, country, university, current_company)

# Create DataFrame
df_generated = pd.DataFrame(dataset)

# Save to Excel file
df_generated.to_excel('generated_dataset_scrapegraphai.xlsx', index=False)


In [ ]:
# Define the generation prompt for ScrapeGraphAI
generation_prompt = """
Generate a dataset with a minimum of 100 records based on the following input criteria:
- Field of activity
- Country
- University
- Current company
"""

# Function to generate dataset based on input criteria
def generate_dataset(field_of_activity, country, university, current_company):
    query = {
        "field_of_activity": field_of_activity,
        "country": country,
        "university": university,
        "current_company": current_company
    }
    response = scrapegraph.run(generation_prompt, query)
    return response

# Example input criteria
field_of_activity = "Software Engineering"
country = "Iran"
university = "University of Tehran"
current_company = "Snapp"

# Generate dataset
dataset = generate_dataset(field_of_activity, country, university, current_company)

# Create DataFrame
df_generated = pd.DataFrame(dataset)

# Save to Excel file
df_generated.to_excel('generated_dataset_scrapegraphai.xlsx', index=False)
